Initializing ConvNext via transfer learning

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
import requests

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# load pre-trained model and configure depth and dimensions
# copied from https://github.com/bamps53/convnext-tf/blob/master/models/convnext_tf.py

model_url = {
    "convnext_tiny_224": "https://github.com/bamps53/convnext-tf/releases/download/v0.1/convnext_tiny_1k_224_ema.h5"
}

model_configs = dict(
    convnext_tiny=dict(
        depths=[3, 3, 9, 3],
        dims=[96, 192, 384, 768]
    )
)

In [3]:
# function to build model (we confine the model to the most basic pre-trained model since we have less classes)
# copied from https://github.com/bamps53/convnext-tf/blob/master/models/convnext_tf.py

def create_model(model_name='convnext_tiny_1k', input_shape=(224, 224), num_classes=10, include_top=True, pretrained=True, use_tpu=False, **kwargs):
    cfg = model_configs['_'.join(model_name.split('_')[:2])]
    print(cfg)
    net = ConvNeXt(num_classes, cfg['depths'], cfg['dims'], include_top, **kwargs)
    net(tf.keras.Input(shape=(*input_shape, 3)))
    if pretrained is True:
        url = model_url
        pretrained_ckpt = tf.keras.utils.get_file(
            f'{model_name}.h5', url, untar=False)
        if use_tpu:
            load_locally = tf.saved_model.LoadOptions(
                experimental_io_device='/job:localhost')
            net.load_weights(
                pretrained_ckpt, options=load_locally, skip_mismatch=True)
        else:
            net.load_weights(pretrained_ckpt, skip_mismatch=True, by_name=True)

    return net

In [4]:
# intiliase model
x = tf.zeros((1, 224, 224, 3), dtype=tf.float32)

model = create_model()
out = model(x) # (1, 1000)

{'depths': [3, 3, 9, 3], 'dims': [96, 192, 384, 768]}


NameError: name 'ConvNeXt' is not defined